# $lookup 的基本操作

## 用法一

In [ ]:
{
   $lookup:
     {
       from: <collection to join>,
       localField: <field from the input documents>,
       foreignField: <field from the documents of the "from" collection>,
       as: <output array field>
     }
}

In [ ]:
# familiarizing with the air_alliances schema
db.air_alliances.findOne()

# familiarizing with the air_airlines schema
db.air_airlines.findOne()

# performing a lookup, joining air_alliances with air_airlines and replacing
# the current airlines information with the new values
db.air_alliances.aggregate([
    {
        $lookup: {
            from: "air_airlines",
            localField: "airlines",
            foreignField: "name",
            as: "airlines"
        }
    }
]).pretty()

In [ ]:
db.air_airlines.aggregate([
    {
        $lookup: {
            from: "air_alliances",
            localField: "name",
            foreignField: "airlines",
            as: "airliance"
        }
    },
    {
        $match: {
            airliance: { $elemMatch: { $exists: true }}
        }
    }
]).pretty()

### 範例01

In [ ]:
db.air_alliances.findOne()
db.air_routes.findOne()

# 要從哪一邊出發很重要
# 注意要記得 $unwind 這一個 stage 
db.air_routes.aggregate([
    {
        $match: {
            airplane: /747|380/
        }
    },
    {
        $lookup: {
            from: "air_alliances",
            localField: "airline.name",
            foreignField: "airlines",
            as: "alliance"
        }
    },
    { $unwind: "$alliance" },
    {
        $group: {
            _id: { alliance: "$alliance.name" },
            num: { $sum: 1 }
        }
    },
    { $sort: { num: -1 }}
]).pretty()

### 範例02

In [ ]:
# 參考解答
db.air_routes.aggregate([
    {
        $match: {
            $or: [ { src_airport: "JFK", dst_airport: "LHR" }, { src_airport: "LHR", dst_airport: "JFK" }]
        }
    },
    {
        $lookup: {
            from: "air_alliances",
            localField: "airline.name",
            foreignField: "airlines",
            as: "air_alliance"
        }
    },
    {
        $match: { air_alliance: { $ne: [] }}
    },
    {
        $group: {
            _id: "$air_alliance.name",
            airlines: { $addToSet: "$airline.name"}
        }
    },
    {
      $addFields: { airlinesNum: { $size: "$airlines" }}   
    },
    {
        $sort: { airlines: -1 }
    }
]).pretty()

In [ ]:
# 參考解答
db.air_routes.aggregate([
  {
    $match: {
      src_airport: { $in: ["LHR", "JFK"] },
      dst_airport: { $in: ["LHR", "JFK"] }
    }
  },
  {
    $lookup: {
      from: "air_alliances",
      foreignField: "airlines",
      localField: "airline.name",
      as: "alliance"
    }
  },
  {
    $match: { alliance: { $ne: [] } }
  },
  {
    $addFields: {
      alliance: { $arrayElemAt: ["$alliance.name", 0] }
    }
  },
  {
    $group: {
      _id: "$airline.id",
      alliance: { $first: "$alliance" }
    }
  },
  {
    $sortByCount: "$alliance"
  }
])

## 用法二

In [ ]:
{
   $lookup:
      {
         from: <joined collection>,
         let: { <var_1>: <expression>, …, <var_n>: <expression> },
         pipeline: [ <pipeline to run on joined collection> ],
         as: <output array field>
      }
}

### 範例01

In [ ]:
# 這題用前面的寫法，有些地方沒辦法在這一個 stage 過濾掉。
db.orders.aggregate( [
   {
      $lookup:
         {
           from: "warehouses",
           let: { order_item: "$item", order_qty: "$ordered" },
           pipeline: [
              { $match:
                 { $expr:
                    { $and:
                       [
                         { $eq: [ "$stock_item",  "$$order_item" ] },
                         { $gte: [ "$instock", "$$order_qty" ] }
                       ]
                    }
                 }
              },
              { $project: { stock_item: 0, _id: 0 } }
           ],
           as: "stockdata"
         }
    }
] )

### 範例02

In [ ]:
db.absences.aggregate( [
   {
      $lookup:
         {
           from: "holidays",
           pipeline: [
              { $match: { year: 2018 } },
              { $project: { _id: 0, date: { name: "$name", date: "$date" } } },
              { $replaceRoot: { newRoot: "$date" } }
           ],
           as: "holidays"
         }
    }
] ).pretty()